In [49]:
import numpy as np #may need numpy
import pandas as pd #may need pandas
import math #need for mathematical calculation
from pprint import pprint #need for showing the results

To calculate the pureness of each subtree we need a measure. Typically, entropy is the way to measure the pureness:

In [124]:
def entropy(probability, summed_probs):
    return - (probability/summed_probs) * math.log(
        probability/summed_probs, 2)

def entropy_of_classes(class_1, class_2):
    #entropy would be 0 if we only have 1 class in our group
    if class_1 == 0 or class_2 == 0:
        return 0
    #otherwise, the sum of the probabilities of the class is the
    #summed_probs and each class has its own probability
    return (entropy(class_1, class_1 + class_2) + 
            entropy(class_2, class_1 + class_2))

#now its time to calculate the whole entropy:
def calculate_entropy(y_hat, y):
    if len(y_hat) != len(y):
        print('Output Error')
        return None
    
    n = len(y)
    s_true = 0
    n_true = len(y[y_hat])
    s_false = 0
    n_false = len(y[~y_hat])
    classes = set(y[y_hat])
    for i in classes:
        num_classes = sum(y[y_hat]==i)
        e = (num_classes/n_true) * entropy_of_classes(
            sum(y[y_hat] == i), sum(y[y_hat] != i))
        s_true += e

    classes = set(y[~y_hat])
    for i in classes:
        num_classes = sum(y[~y_hat] == i)
        e = (num_classes/n_false) * entropy_of_classes(
            sum(y[~y_hat] == i), sum(y[~y_hat] != i))
        s_false += e
    
    s = n_true / n * s_true + n_false / n * s_false
    
    return s

In [141]:
class decision_tree_classifier(object):
    def __init__(self, max_depth):
        self.depth = 0 #we want to count the depth we are going through
        self.max_depth = max_depth #this values has been gotten as input
    
####*******************************************************************************#############   
    
    def find_best_split_of_all(self, x, y):
        feature = None
        n = len(y)
        min_entropy = 1
        max_val = None
        
        for bestFeature, feature in enumerate(x.T):
            entropy = 10    
            for value in set(feature):
                y_hat = feature < value
                current_entropy = calculate_entropy(y_hat, y)  # get entropy of this split
                if current_entropy <= entropy:
                    entropy = current_entropy
                    curr_maxVal = value
            
            if entropy == 0:
                return bestFeature, curr_maxVal, entropy
            
            elif entropy <= min_entropy:
                min_entropy = entropy
                best_feature = bestFeature
                max_val = curr_maxVal
                
        return best_feature, max_val, min_entropy

####*******************************************************************************#############   
    
    #this function is solely for fitting the data to a model to find the best
    #hopothysis among existing hopothysises
    def fit(self, x, y, depth = 0):
        dict_of_nodes = {} #we need a dictionary to save the partitions
        depth=0 #keep track of depth we go through

        #when dictionary is null, then we do not need to go forward
        #so we return None
        if dict_of_nodes is None:
            return None
        
        #When we have no data for a single group, we cannot go forward
        elif len(y) == 0:
            return None
        
        #when all y we have are from the same group, we return its value
        elif all(i == y[0] for i in y):
            return {'value': y[0]}
        
        #if depth is reached its maximum, we stop the progression
        elif depth >= self.max_depth:   
            return None
        
        # If none of the above, we resume the ID3 algorithm
        else:   
            #we split the data based on its information gain:
            feature, max_val, entropy = self.find_best_split_of_all(x, y)
            
            #we separate data of the class which are less than the max_value
            #on the left subtree
            true_values = x[:, feature] < max_val #gives us [True or False] array
            y_left = y[true_values] #converts the [True or False] array into [1 or 0]

            #the same as above but for larger than the max_value goes to the right hand side
            true_values = x[:, feature] >= max_val
            y_right = y[true_values]
            
            dict_of_nodes = {
                'feature': iris.feature_names[feature],
                'index_feature':feature, 
                'max_value':max_val,
                'value': np.round(np.mean(y))
            }
            
            # recursively we generate tree for the left hand side
            dict_of_nodes['left'] = self.fit(x[x[:, feature] < max_val], y_left, depth+1)
            
            # recursively we generate tree for the right hand side
            dict_of_nodes['right'] = self.fit(x[x[:, feature] >= max_val], y_right, depth+1)  
            
            self.depth += 1 #increasing the depth after coming out of the recursion
            self.trees = dict_of_nodes  
            return dict_of_nodes

####*******************************************************************************#############   
        
    def predict(self, x):
        results = np.zeros(len(x))        
        for i, c in enumerate(x):
            cur_layer = self.trees
            while cur_layer.get('max_value'):
                if c[cur_layer['index_feature']] < cur_layer['max_value']:
                    cur_layer = cur_layer['left']
                else:
                    cur_layer = cur_layer['right']
            else:
                results[i] = cur_layer.get('value')

        return results

In [155]:
from sklearn.datasets import load_iris
iris = load_iris()
x = iris.data
y = iris.target

In [156]:
len(x) #not much data

150

In [157]:
x_train = x[:120]
y_train = y[:120]
x_test = x[120:]
y_test = y[120:]

In [166]:
clf = decision_tree_classifier(max_depth=7)
vision = clf.fit(x_train, y_train)
clf.predict(x_test)

array([2., 2., 2., 1., 2., 2., 1., 1., 2., 1., 2., 2., 2., 1., 2., 2., 2.,
       2., 1., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2.])

Comparing With SKLearn

In [167]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
clf.predict(x_test)

array([2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

Lets See how it looks like:

In [168]:
pprint(vision)

{'feature': 'petal width (cm)',
 'index_feature': 3,
 'left': {'value': 0},
 'max_value': 1.0,
 'right': {'feature': 'petal width (cm)',
           'index_feature': 3,
           'left': {'feature': 'petal length (cm)',
                    'index_feature': 2,
                    'left': {'value': 1},
                    'max_value': 5.0,
                    'right': {'feature': 'sepal width (cm)',
                              'index_feature': 1,
                              'left': {'value': 2},
                              'max_value': 2.7,
                              'right': {'value': 1},
                              'value': 2.0},
                    'value': 1.0},
           'max_value': 1.7,
           'right': {'feature': 'petal length (cm)',
                     'index_feature': 2,
                     'left': {'feature': 'sepal length (cm)',
                              'index_feature': 0,
                              'left': {'value': 2},
                             